In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

# 1. 數據準備 (沿用之前的邏輯，或是直接 copy 過去)
tickers = ['TSM', 'SPY', 'NVDA', 'AAPL']

# ... (請自己補上抓數據、算 mean_returns 和 cov_matrix 的代碼) ...
data = yf.download(tickers, start='2020-01-01', end='2023-01-01')['Close']
returns = data.pct_change().dropna()    
mean_returns = returns.mean()
cov_matrix = returns.cov()

# 2. 定義目標函數 (Objective Function)
# 我們要最小化「負的夏普比率」 (Negative Sharpe Ratio)，因為 minimize 只能找最小值
# Sharpe = (R - Rf) / Volatility
# 假設無風險利率 Rf = 0

def negative_sharpe(weights, mean_returns, cov_matrix, risk_free_rate=0.0):
    # TODO 1: 算出年化報酬 (scalar)
    # p_ret = ... (記得 * 252)
    p_ret = np.sum(weights * mean_returns) * 252
    
    # TODO 2: 算出年化風險 (scalar)
    # p_vol = ... (記得 sqrt(w @ Sigma @ w.T) * sqrt(252))
    p_vol = np.sqrt(weights @ cov_matrix @ weights.T) * np.sqrt(252)

    # TODO 3: 算出夏普比率並取負號
    # return - (p_ret - risk_free_rate) / p_vol
    return - (p_ret - risk_free_rate) / p_vol if p_vol else 0

tickers = returns.columns.tolist()

# 3. 設定約束條件 (Constraints & Bounds)
num_assets = len(tickers)
args = (mean_returns, cov_matrix)

# 權重總和為 1: sum(w) - 1 = 0
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# 每個權重在 0 到 1 之間 (不允許放空)
bounds = tuple((0.0, 1.0) for asset in range(num_assets))

# 初始猜測 (平分)
init_guess = num_assets * [1. / num_assets,]

# 4. 執行優化 (實驗 A: 原始數據)
print("--- 實驗 A: 原始數據最佳權重 ---")
result_A = minimize(negative_sharpe, init_guess, args=args,
                  method='SLSQP', bounds=bounds, constraints=constraints)

weights_A = result_A.x
# 顯示權重 (保留小數點後 4 位)
print(pd.Series(weights_A, index=tickers).round(4))


# 5. 執行優化 (實驗 B: 蝴蝶效應)
# 任務：創造一個 modified_mean_returns
# 讓 TSM 的預期報酬率「微幅增加」 1% (原本可能是 0.002，變成 0.002 * 1.01 或 + 0.0001)
# 注意：只改這一點點！

# TODO 4: 複製 mean_returns 並修改 TSM 的值
# modified_mean_returns = ...
modified_mean_returns = mean_returns.copy()
modified_mean_returns['TSM'] += 0.005

print(f"\nOriginal TSM mean: {mean_returns['TSM']:.6f}")
print(f"Modified TSM mean: {modified_mean_returns['TSM']:.6f}")

print("\n--- 實驗 B: 修改 TSM 預期報酬 (+1%) 後的權重 ---")
# TODO 5: 用 modified_mean_returns 再跑一次 minimize，存入 weights_B
# result_B = ...
# weights_B = ...
result_B = minimize(negative_sharpe, init_guess, args=(modified_mean_returns, cov_matrix),
                  method='SLSQP', bounds=bounds, constraints=constraints)
weights_B = result_B.x

print(pd.Series(weights_B, index=tickers).round(4))

# 6. 比較差異
print("\n--- 敏感度分析 ---")
diff = weights_B - weights_A
print(pd.Series(diff, index=tickers).round(4))


[*********************100%***********************]  4 of 4 completed

--- 實驗 A: 原始數據最佳權重 ---
AAPL    0.3646
NVDA    0.6354
SPY     0.0000
TSM     0.0000
dtype: float64

Original TSM mean: 0.000685
Modified TSM mean: 0.005685

--- 實驗 B: 修改 TSM 預期報酬 (+1%) 後的權重 ---
AAPL    0.0
NVDA    0.0
SPY     0.0
TSM     1.0
dtype: float64

--- 敏感度分析 ---
AAPL   -0.3646
NVDA   -0.6354
SPY    -0.0000
TSM     1.0000
dtype: float64
